# ***Interactive*** Titanic Notebook

<img src="https://media.giphy.com/media/g2e22SIHPcQw0/giphy.gif">

### Hi there! This is my first Kaggle notebook, and instead of doing the cliché basic scripts or highly advanced ML scripts (since I am not an expert), I decided to add some fun interactive plots and visualizations to showcase the Titanic dataset and evaluate some generic algorithms. 

`Aesthetics are a moral imperative`

> ## Quick rundown:
* [Starting housekeeping](#first-section)
* [Data Visualization](#second-section)
* [Evaluation of ML Models](#third-section)
* [Submission](#fourth-section)

## Starting housekeeping<a class="anchor" id="first-section"></a>

### Importing tools, data, and cleaning up the data.

In [ ]:
# Let's start off importing some python tools
import pandas as pd
import numpy as np
import random as rnd
from tabulate import tabulate

# Visualization libraries 
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import plotly as py
import plotly.express as px
import plotly.graph_objects as go

#!pip install heatmapz # Really great heatmap visualization from https://pypi.org/project/heatmapz/
#from heatmap import heatmap, corrplot
%matplotlib inline
#%matplotlib notebook

# ML tools
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn import svm
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB,CategoricalNB,MultinomialNB
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, accuracy_score
from xgboost import XGBClassifier
from sklearn.metrics import jaccard_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss

# Importing Datasets to pandas DataFrames
train_df = pd.read_csv("/kaggle/input/titanic/train.csv")
test_df = pd.read_csv("/kaggle/input/titanic/test.csv")
complete_df = [train_df, test_df] #Helpful when cleaning both dataframes at once

# Completing or deleting missing values in the dataset
# I am filling in median age, mode embark, and mediam fare for missing values
for datadf in complete_df:    
    #complete missing age with median
    datadf['Age'].fillna(datadf['Age'].median(), inplace = True)

    #complete embarked with mode
    datadf['Embarked'].fillna(datadf['Embarked'].mode()[0], inplace = True)

    #complete missing fare with median
    datadf['Fare'].fillna(datadf['Fare'].median(), inplace = True)
    
# Getting rid of irrelevant columns: Passanger ID, Cabin number, and Ticker number
drop_column = ['PassengerId','Cabin', 'Ticket']
for datadf in complete_df:    
    datadf.drop(drop_column, axis=1, inplace = True)
    
# Ok, time to visualize the data!
import warnings
import numpy as np
warnings.simplefilter(action='ignore', category=FutureWarning)
print('x' in np.arange(5))   #returns False, without Warning

> ## Data Visualization <a class="anchor" id="second-section"></a>

### For distribution of data, statistical analysis is much more powerful visually than in a giant chart. In the first plot, I am using simple "*seaborn*" functions to combine a swarmplot and boxplot to highlight the age and sex distribution. Individual people are colorcoded to show if they survived, as this parameter is our target. Box plot is shown around the datapoints. 

### Sometimes simple is best.

In [ ]:
# General age and sex distribution with added swarmplot
plt.figure(figsize=(12,8))

# sns plot would add a default legend of 0 or 1 for Survived column, so change it to "Died" and "Survived"
custom = [Line2D([], [], marker='o', color='#023EFF', linestyle='None'),
          Line2D([], [], marker='o', color='#FF7C00', linestyle='None')]
sns.swarmplot(x="Age", y="Sex", hue="Survived", data=train_df, palette="bright")
ax = sns.boxplot(x="Age", y="Sex", data=train_df, color='white')

# Making the boxplot edge colors black for better contrast
for i,box in enumerate(ax.artists):
    box.set_edgecolor('black')
    box.set_facecolor('white')
    # iterate over whiskers and median lines
    for j in range(6*i,6*(i+1)):
         ax.lines[j].set_color('black')
            
# Setting the text for legend, overall font size, and title of figure            
plt.legend(custom, ['Died', 'Survived'], loc='upper right')
plt.rc('font', size=20)
plt.rc('axes', titlesize=20)

# This plot gives a nice view of the phrase "Women and Children first!"

In [ ]:
# Let's see how port of embarkation and fare correlate
plt.figure(figsize=(12,8))

# sns plot would add a default legend of 0 or 1 for Survived column, so change it to "Died" and "Survived"
custom = [Line2D([], [], marker='o', color='#023EFF', linestyle='None'),
          Line2D([], [], marker='o', color='#FF7C00', linestyle='None')]
sns.swarmplot(x="Fare", y="Embarked", hue="Survived", data=train_df, palette="bright")
ax = sns.boxplot(x="Fare", y="Embarked", data=train_df, color='white')

# Making the boxplot edge colors black for better contrast
for i,box in enumerate(ax.artists):
    box.set_edgecolor('black')
    box.set_facecolor('white')
    # iterate over whiskers and median lines
    for j in range(6*i,6*(i+1)):
         ax.lines[j].set_color('black')
            
# Setting the text for legend, overall font size, and title of figure            
plt.legend(custom, ['Died', 'Survived'], loc='upper right')
plt.rc('font', size=20)
plt.rc('axes', titlesize=20)

### For more interactive plots, "*plotpy*" express library provides quick and simple visualization. A 2D scatter plot may also contain built in histograms or box plots. By clicking on the legend marker groups, you can select to display only certain results. Hovering over individual points yields the full parameters for each point.

In [ ]:
train_df["Survived"] = train_df["Survived"].astype(str)
fig = px.scatter(train_df, x="Age", y="Fare", color="Survived",hover_data=['SibSp','Parch','Pclass','Embarked'],
                 marginal_x="box", marginal_y="box" ,title="Box plots of Age and Fare",hover_name='Name')
fig.update_layout(legend_title_text='Survived? 0=No, 1=Yes')
fig

### How does the passenger distribution vary for the three Ticket Classes (1st, 2nd, 3rd)? Pull slider to find out! The colors help distinguish port of embarkation. (Survived 0=No, 1=Yes)

In [ ]:
train_df = train_df.sort_values(by=['Pclass'])
fig = px.scatter(train_df, x="Age", y="Fare",animation_frame="Pclass",
                 color="Embarked", hover_name="Name", facet_col="Survived",
           title="Scatter Plots of Age and Fare",log_x=False, size_max=45)
fig.show()

### Can we visualize the data in 3d? This would make more sense with other types of data, but we can try with Titanic passengers. Try moving the 3d plot around with your mouse. 

### Here, the size of the data point is total family members (siblings + spouse + children + parents)

In [ ]:
train_df["Survived"] = train_df["Survived"].astype(str)
train_df = train_df.sort_values(by=['Survived'])
fig = px.scatter_3d(train_df, x='Age', y='Fare', z='Pclass', size = train_df['SibSp']+train_df['Parch'],
              color='Survived', size_max=40,
              symbol='Sex', opacity=0.8,hover_name="Name",hover_data=['SibSp','Parch','Embarked'])
# tight layout
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))

### How can we see number of family members of the passengers? Here, z axis is total family members (siblings + spouse + children + parents)

In [ ]:
# 3D plot based on Age, Fare, and Family size (siblings+spouse+children+parents in z)
train_df["Survived"] = train_df["Survived"].astype(str)
train_df = train_df.sort_values(by=['Survived'])

fig = px.scatter_3d(train_df, x='Age', y='Fare', z=train_df['SibSp']+train_df['Parch'],
              color='Survived', size_max=10,
              symbol='Sex', opacity=0.7,hover_name="Name",hover_data=['SibSp','Parch','Embarked'])
# tight layout
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))

## Evaluation of ML Models <a class="anchor" id="third-section"></a>

### Before we try various models, the data needs some additional pre-processing. Specifically, we should covert categorical features to numerical values (Sex, Embarked).

In [ ]:
train_df['Sex'].replace(to_replace=['male','female'], value=[0,1],inplace=True)
train_df['Embarked'].replace(to_replace=['C','Q', 'S'], value=[0,1,2],inplace=True)

test_df['Sex'].replace(to_replace=['male','female'], value=[0,1],inplace=True)
test_df['Embarked'].replace(to_replace=['C','Q', 'S'], value=[0,1,2],inplace=True)

### Feature (X) and label (y) selection. Then normalization of data to give data zero mean and unit variance. X_submit is the data for final submission. 

In [ ]:
X = train_df
y = train_df['Survived'].values
X.drop(['Name'],axis = 1, inplace = True)
X.drop(['Survived'],axis = 1, inplace = True)

X_submit = test_df
X_submit.drop(['Name'],axis = 1, inplace = True)

X = preprocessing.StandardScaler().fit(X).transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

### Decision Tree

In [ ]:
# From sklearn.tree import DecisionTreeClassifier
DT_model = DecisionTreeClassifier(criterion="entropy")
DT_model.fit(X_train,y_train)

DT_yhat = DT_model.predict(X_test)

print("DT accuracy: %.2f" % accuracy_score(y_test, DT_yhat))
print("DT Jaccard index: %.2f" % jaccard_score(y_test, DT_yhat,pos_label='1'))
print("DT F1-score: %.2f" % f1_score(y_test, DT_yhat, average='weighted') )

### Random Forest Classifier

In [ ]:
RanFor_model = RandomForestClassifier(n_estimators=10,random_state=1).fit(X_train,y_train)
RanFor_yhat = RanFor_model.predict(X_test)

print("Random Forest accuracy: %.2f" % accuracy_score(y_test, RanFor_yhat))
print("Random Forest Jaccard index: %.2f" % jaccard_score(y_test, RanFor_yhat,pos_label='1'))
print("Random Forest F1-score: %.2f" % f1_score(y_test, RanFor_yhat, average='weighted') )

### Support Vector Machine

In [ ]:
# From sklearn import svm.SVC()
SVM_model = SVC()
SVM_model.fit(X_train,y_train)

SVM_yhat = SVM_model.predict(X_test)

print("SVM accuracy: %.2f" % accuracy_score(y_test, SVM_yhat))
print("SVM Jaccard index: %.2f" % jaccard_score(y_test, SVM_yhat,pos_label='1'))
print("SVM F1-score: %.2f" % f1_score(y_test, SVM_yhat, average='weighted') )

### Logistic Regression

In [ ]:
LR_model = LogisticRegression(C=0.01).fit(X_train,y_train)
LR_yhat = LR_model.predict(X_test)

print("LR accuracy: %.2f" % accuracy_score(y_test, LR_yhat))
print("LR Jaccard index: %.2f" % jaccard_score(y_test, LR_yhat,pos_label='1'))
print("LR F1-score: %.2f" % f1_score(y_test, LR_yhat, average='weighted') )

### Bernoulli Naive Bayes

In [ ]:
NB_model = BernoulliNB(2).fit(X_train,y_train)
NB_yhat = NB_model.predict(X_test)

print("NB accuracy: %.2f" % accuracy_score(y_test, NB_yhat))
print("NB Jaccard index: %.2f" % jaccard_score(y_test, NB_yhat,pos_label='1'))
print("NB F1-score: %.2f" % f1_score(y_test, NB_yhat, average='weighted') )

### XGBClassifier

In [ ]:
XGB_model=XGBClassifier(max_depth=3).fit(X_train,y_train)
XGB_pred=XGB_model.predict(X_test)
    
print("XGB accuracy: %.2f" % accuracy_score(y_test, XGB_pred))
print("XGB Jaccard index: %.2f" % jaccard_score(y_test, XGB_pred,pos_label='1'))
print("XGB F1-score: %.2f" % f1_score(y_test, XGB_pred, average='weighted') )    
    

### Model comparison:

<img src="https://imgs.xkcd.com/comics/machine_learning.png">

In [ ]:
from tabulate import tabulate
data = [['Decision Tree', accuracy_score(y_test, DT_yhat), jaccard_score(y_test, DT_yhat,pos_label='1'), f1_score(y_test, DT_yhat, average='weighted')],
['Random Forest Classifier', accuracy_score(y_test, RanFor_yhat), jaccard_score(y_test, RanFor_yhat,pos_label='1'), f1_score(y_test, RanFor_yhat, average='weighted')],
['Support Vector Machine', accuracy_score(y_test, SVM_yhat), jaccard_score(y_test, SVM_yhat,pos_label='1'), f1_score(y_test, SVM_yhat, average='weighted')],
['Logistic Regression', accuracy_score(y_test, LR_yhat), jaccard_score(y_test, LR_yhat,pos_label='1'), f1_score(y_test, LR_yhat, average='weighted')],
['Bernoulli Naive_Bayes', accuracy_score(y_test, NB_yhat), jaccard_score(y_test, NB_yhat,pos_label='1'), f1_score(y_test, NB_yhat, average='weighted')],
['XGB Classifier', accuracy_score(y_test, XGB_pred), jaccard_score(y_test, XGB_pred,pos_label='1'), f1_score(y_test, XGB_pred, average='weighted')]]
print (tabulate(data, headers=["Model", "Accuracy", "Jaccard score", "F1-Score"]))

### Highest accuracy and F1-score were obtained from Random Forest and XGB Classifier.

## Submission <a class="anchor" id="fourth-section"></a>

In [ ]:
prediction_submit = RanFor_model.predict(X_submit)

testdata = pd.read_csv("/kaggle/input/titanic/gender_submission.csv")
output = pd.DataFrame({'PassengerId': testdata.PassengerId, 'Survived': prediction_submit})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")

### *Note that additional data and correlations may have been extracted from the Name column by parsing the title, first and last name. For example, the "Mrs." and "Miss" title would imply a woman being married or unmarried. However, I did not go into such detail.
    